## Querying Relational Databases

- Databases usually have multiple tables
- Tables have different but related data
- Often reference or relate to each other by a key
- Make connections by **JOIN**ing


## movielens data set

- Full dataset: [grouplens.org/datasets/movielens](http://grouplens.org/datasets/movielens/)
    - 27,000 movies
    - 470,000 tags
    - 21,000,000 ratings
    - by 230,000 users - anonymized
- We'll be working with a subset in SQLite format **movielens-small**

## Reading the data

Open the **querying-movielens.ipynb** notebook from the [day2-data.zip file](http://cliburn.github.io/cfar-data-workshop-2015/)



In [3]:
from sqlite_utils import SQLiteDatabase
ml = SQLiteDatabase('movielens-small.db')

In [ ]:
# Know your limits
ml.query("SELECT * FROM movies")

### Review on row order

- No inherent order in the rows - reference them by a unique value and not a position.
- movieId is a **PRIMARY KEY**. It's an identifier that must be different for every movie
- Can order by multiple columns, let's try year then title...


### Exercise:

Query (`SELECT`) the **movies** table, sorting by **year** (newest first) then **title** (A-Z)

`... ORDER BY <column 1> (ASC|DESC), <column 2> (ASC|DESC)`

What's the first movie listed? The last?

In [ ]:
# movies by year (newest) then by title (a-z)
ml.query('')

In [ ]:
# Continue exploring - ratings?

In [ ]:
ml.query("SELECT * FROM ratings")

## Ratings table

- The rating itself is a number (e.g. number of stars)
- Each rating has a **userId**, **movieId**, and **timestamp**
    - Timestamp is seconds since Jan 1, 1970


In [ ]:
# Can convert with python
import time
time.ctime(980730861)

In [ ]:
# Or in SQL:
ml.query("SELECT *, datetime(timestamp, 'unixepoch') as date from ratings limit 3")

### Relationship to movies
- We have a **movies** table also with a **movieId**
- We can look up ratings for a movie, but we need to know its **movieId**

That common **movieId** column is a **KEY**

- In the **movies** table, it's a **PRIMARY KEY**
- In the **ratings** table, it's a **FOREIGN KEY**

## Exercise: Relational Data

Look up a favorite movie and find newest and oldest rating.

Tips:
- Titles like **The Godfather** are listed as **Godfather, The**
- Sorting the **timestamp** column can be used to find the oldest (ASC) or newest (DESC)
- Can use `datetime(timestamp, 'unixepoch')`

In [ ]:
ml.query("SELECT * FROM ratings WHERE _ = _ ORDER BY _ DESC")

### Aggregate Functions

- Not practical to query/scan individual ratings
- Even with Pandas, you may not want hold the dataset in memory. Ask for what you want.
- Functions like average, count, can be part of the query.

[SQLite Aggregate Functions](https://www.sqlite.org/lang_aggfunc.html)

- avg
- count
- max / min
- sum
- group_concat 

In [ ]:
ml.query("SELECT COUNT(*) FROM ratings WHERE movieId = ___")
# How many ratings does your movie have?
# What's the average rating for your movie?

### Distinct

Distinct is often used with these aggregate functions, especially with values that repeat across rows, like a year.

In [ ]:
 # How many years in the table? What's the max and min?
ml.query("SELECT COUNT(year) FROM movies")

In [ ]:
# Seems like a lot, what's the span?
ml.query("SELECT year FROM movies limit 10")

### Exercise: Aggregates

The full dataset has:
- 27,000 movies
- 470,000 tags
- 21,000,000 ratings
- by 230,000 users


1. What are these statistics for the tables in your database?
2. What's the average rating for all movies?
3. If each rating is a star, how many **total** stars have been given for **Shawshank Redemption, The**
4. What's the date of the oldest rating? The newest?

Hint:
- No users table but userIds exist in ratings


# Querying Multiple Tables

Joins and keys

The real power of relational databases is in the **relations**.

movies and ratings are related, but there are other tables:

- movies:links is 1:1
- movies:ratings 1:Many
- movies:tags is Many:Many

Connect related tables in a single query, with a **JOIN**

### JOIN Query

Let's have a look at the links table, and see how we can get the IMDB link for some movies in the table.

In [ ]:
# Start with SELECT movies.title FROM movies
# JOIN links ON movies.movieId = links.movieId

In [ ]:
movies_links = '''
SELECT movies.title, links.*
FROM movies
JOIN links on movies.movieId = links.movieId
'''
ml.query(movies_links)

### Exercise

Write a query that JOINs movies and links to print titles and links for each of the Star Wars movies

In [ ]:
starwars_links = '''
SELECT 
FROM
JOIN
WHERE
'''
ml.query(starwars_links)

In [ ]:
starwars_links = '''
SELECT
    movies.title,
    links.*
FROM movies
JOIN links on movies.movieId = links.movieId
WHERE movies.title like 'Star Wars%'
ORDER BY movies.title asc
'''
ml.query(starwars_links)

### Joins and Aggregates

We can join tables and we can run aggregate functions like AVG/COUNT, useful to put those together.

Average rating for all the star wars movies

In [ ]:
# Start with a query, then add the aggregates
ratings_query = '''
SELECT ratings.rating
FROM movies
JOIN ratings ON movies.movieId = ratings.movieId
WHERE movies.title like 'Star Wars: Episode%'
'''
ml.query(ratings_query)

### Exercise

Starting with the average ratings of Star Wars query:

1. How many ratings? How many total stars?
2. How does the average rating of the first 3 (before 1999) compare to the second 3 (after 1999)?
4. How does the average rating of these movies compare to the Godfather trilogy?


## Grouping

When aggregating, often makes sense aggregate groups of data instead of whole dataset

For example: average rating for each movie instead of all movies

This is called grouping, we covered it in Pandas, and it's also possible in SQL.


In [ ]:
ml.query('select year, count(*) FROM movies GROUP BY year')

### Exercise: Grouping with Joins

Grouping works with any query, not just single-tables. 

Let's apply this to the ratings query from earlier, and see what that looks like

1. Include title (I'll get you started)
2. Include year
3. Include number of ratings as **num_ratings**
4. Order by the average rating (highest first)
5. Order by the most ratings (highest first)

In [ ]:
ratings_query = '''
SELECT avg(ratings.rating) AS avg_rating
FROM movies
JOIN ratings ON movies.movieId = ratings.movieId
WHERE movies.title like 'Star Wars: Episode%'
'''
ml.query(ratings_query)

### 5 Highest-Rated Movies

Let's write a query to find the 5 highest rated movies

We're pretty close, just need to remove the WHERE clause and add a LIMIT, right?

In [ ]:
# 5 highest rated movies
ratings_query = '''
SELECT 
    movies.title, 
    avg(ratings.rating) AS avg_rating
FROM movies
JOIN ratings ON movies.movieId = ratings.movieId
GROUP BY movies.title 
ORDER BY avg_rating DESC
LIMIT 5
'''
ml.query(ratings_query)

### HAVING
- A WHERE clause for aggregates


### Exercises

- Get movies with over 100 ratings and at least a 4 star average
- Get movies with at least 15 tags
- Include those tags with group_concat function

In [ ]:
ratings_query = '''
SELECT 
    movies.title, 
    movies.year,
    count(tags.tag) AS num_tags
FROM
    movies
JOIN
    tags ON movies.movieId = tags.movieId
GROUP BY
    movies.title
order by year
'''
ml.query(ratings_query)

### Putting it all together

- Plotting avg score by year



In [9]:
# Imports for pandas, sqlite, and plt
import pandas as pd
from pandas import DataFrame as df
import sqlite3
import matplotlib.pyplot as plt

In [ ]:
# Build up the query as a string
ratings_query = '''
SELECT 
    movies.year,
    avg(ratings.rating) as avg_rating,
    count(ratings.rating) as num_ratings
FROM
    movies
JOIN
    ratings ON movies.movieId = ratings.movieId
GROUP BY
    movies.year
order by year
'''

# Create a Data Frame from the query
conn = sqlite3.connect('movielens-small.db')
ratings_by_year = pd.read_sql_query(ratings_query, conn)
conn.close()


In [ ]:
# Tell Matplotlib to plot inline
%matplotlib inline

# Generate some plots from the data frame
ratings_by_year.plot(kind='line', x='year', y='num_ratings', xlim=(1900,2020), figsize=(16,4))
plt.xlabel('Year')
plt.ylabel('Number of Ratings')

ratings_by_year.plot(kind='scatter', x='year', y='avg_rating', xlim=(1900,2020), ylim=(0,5), figsize=(16,4))
plt.xlabel('Year')
plt.ylabel('Average Rating')

pass

In [ ]:
ratings_by_year.info()

In [ ]:
big_query = '''
SELECT 
    movies.year,
    ratings.rating
FROM
    movies
JOIN
    ratings ON movies.movieId = ratings.movieId
order by year'''
conn = sqlite3.connect('movielens-small.db')
all_ratings_and_year = pd.read_sql_query(big_query, conn)
conn.close()

all_ratings_and_year.info()